# Instrucciones para usar el robot real

Cómo usar el robot real de manera remota y qué sensores tiene.

**Autor:** _Ing. Emiliano J. Borghi Orué_ (eborghiorue@frba.utn.edu.ar) 

![Banner](media/banner.png)


## Información general

iRobot Create 2 es un robot del tipo diferencial.

### Robot diferencial

<img src="media/diff-drive.png" />

#### Eq. cinemáticas

\begin{align}

V_{R} = \omega \cdot ( R + \frac{L}{2} ) \\
V_{L} = \omega \cdot ( R - \frac{L}{2} )

\\\\

V_{R} = r \cdot \omega_{R} \\
V_{L} = r \cdot \omega_{L}

\\\\

v = \frac{V_{R} + V_{L}}{2} \\
\omega = \frac{V_{R} - V_{L}}{L}

\\\\

\begin{bmatrix}
\dot{x} \\ 
\dot{y} \\ 
\dot{\theta}
\end{bmatrix}

=

\begin{bmatrix}
\cos{ \theta } && 0 \\ 
\sin{ \theta } && 0 \\ 
0 && 1
\end{bmatrix}

\cdot

\begin{bmatrix}
v \\ \omega
\end{bmatrix}

\end{align}

### Sensores del robot

- Nivel de batería
- Sensor IR omnidireccional para dockeo
- Sensor de precipicios x 4 (cliff)
- Encoders x2
- Wheel drop x2
- Sensor de suciedad
- Bumpers
- IMU
- LiDAR
- Cámara
- ...

### Open Interface

El robot ofrece la posibilidad de comunicarse con el robot a través de su puerto serie.
Nosotros no usaremos este tipo de comunicación directamente, pero para saber más, el siguiente documento ofrece toda la información necesaria:

In [ ]:
from IPython.display import IFrame
IFrame("./media/iRobot_Roomba_600_Open_Interface_Spec.pdf", width=512, height=768)

### libcreate y create_autonomy
Nosotros no usaremos lo anterior para comandar al robot de manera directa debido a que eso ya está implementado en el paquete de ROS [`libcreate`](https://github.com/RoboticaUtnFrba/libcreate).
Para comandar al robot se usarán tópicos de ROS, los cuales son provistos por el metapaquete [`create_autonomy`](https://github.com/RoboticaUtnFrba/create_autonomy); más específicamente el paquete [`ca_driver`](https://github.com/RoboticaUtnFrba/create_autonomy/tree/kinetic-devel/ca_driver) se conecta a `libcreate` para leer el hardware y crear una interfaz de ROS.

### [Conectándose al robot](https://github.com/RoboticaUtnFrba/create_autonomy/wiki/Remote-connection)
Si se quiere conectar a la Nvidia Jetson Nano que controla al robot de manera remota, hay que realizar dos pasos:
- Conectarse a la red WiFi **RoboticaUtnFrba**. Pedir la contraseña a los docentes de la cátedra.
- Utilizar SSH con el siguente comando:

In [ ]:
ssh USER@PC_NAME.local

Reemplazar los datos `USER` y `PC_NAME` están anotados en una etiqueta en el robot.
También será necesario introducir una contraseña para conectarse al robot, el cual también será provisto por la cátedra.

#### [Primeras pruebas](https://github.com/RoboticaUtnFrba/create_autonomy/wiki/Getting-started#launch-files)
La Nvidia Jetson Nano es una computadora como cualquier otra con Ubuntu 18.04.
Para rápidamente probar el funcionamiento del robot se puede compilar el repositorio (en el branch que deseemos), sourcear, y ejecutar el launchfile deseado:

In [ ]:
cd /create_ws/src/create_autonomy
git fetch origin BRANCH_NAME
git checkout origin BRANCH_NAME
cd /create_ws
catkin_make -DCMAKE_BUILD_TYPE=Release
source devel/setup.bash
roslaunch ca_driver create_2.launch

Como podemos ver, y se ha mencionado previamente, se está ejecutando únicamente el launchfile que interface directamente con `libcreate`.

#### [Obtener datos directamente a tu computadora](https://github.com/RoboticaUtnFrba/create_autonomy/wiki/Remote-connection)
ROS permite que cualquier dispositivo en la misma red comparta el Máster y así ejecutar nodos de manera remota.
Para que esto sea posible, hay que hacer un cambio a qué se ejecuta desde el robot:

In [ ]:
roscd ca_bringup/scripts
source robot_network_config.sh
roslaunch ca_bringup minimal.launch raspicam_receiver_IP:=<COMPUTER_IP>

Estos pasos anteriores cambias algunas variables de entorno de ROS para habilitar lo mencionado previamente.
**Nota:** Reemplazar `<COMPUTER_IP>` por la IP de la Nvidia Jetson Nano, la cual puede obtenerse dentro de la misma con este comando:

In [ ]:
hostname -I

Ahora, desde la computadora, hay que ejecutar los siguientes comandos:

In [ ]:
roscd ca_bringup/scripts
source remote_access_config.sh <ROBOT_IP>

La IP del robot es necesaria también aquí.
Una vez hecho esto, podrás ver toda la información del robot. Por ejemplo:

In [ ]:
rostopic list

#### [Teleoperando al robot](https://github.com/RoboticaUtnFrba/create_autonomy/wiki/Getting-started#teleoperation)
Al robot se lo puede teloperar con un joystick o el teclado de la PC:
- Joystick:

In [ ]:
roslaunch ca_tools joy_teleop.launch

- Teclado:

In [ ]:
roslaunch ca_tools keyboard_teleop.launch